# Project

### Import Modules

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

### Loading Datasets

In [3]:
FILENAME= "calendar.csv"
DATADIR = "../pfda_project/"

calendar_df = pd.read_csv(DATADIR + FILENAME, header=0)

In [4]:
FILENAME= "inventory.csv"
DATADIR = "../pfda_project/"

inventory_df = pd.read_csv(DATADIR + FILENAME, header=0)

In [5]:
FILENAME= "sales_test.csv"
DATADIR = "../pfda_project/"

sales_test_df = pd.read_csv(DATADIR + FILENAME, header=0)

In [6]:
FILENAME= "sales_train.csv"
DATADIR = "../pfda_project/"

sales_train_df = pd.read_csv(DATADIR + FILENAME, low_memory=False)

In [7]:
FILENAME= "solution.csv"
DATADIR = "../pfda_project/"

solution_df = pd.read_csv(DATADIR + FILENAME)

In [8]:
FILENAME= "test_weights.csv"
DATADIR = "../pfda_project/"

test_weights_df = pd.read_csv(DATADIR + FILENAME)

### Information about datasets

##### calendar_df

In [9]:
calendar_df.head()

,date,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays,warehouse
0,2022-03-16,NaN,0,0,0,0,Frankfurt_1
1,2020-03-22,NaN,0,0,0,0,Frankfurt_1
2,2018-02-07,NaN,0,0,0,0,Frankfurt_1
3,2018-08-10,NaN,0,0,0,0,Frankfurt_1
4,2017-10-26,NaN,0,0,0,0,Prague_2


In [10]:
calendar_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23016 entries, 0 to 23015
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   date                    23016 non-null  object
 1   holiday_name            930 non-null    object
 2   holiday                 23016 non-null  int64 
 3   shops_closed            23016 non-null  int64 
 4   winter_school_holidays  23016 non-null  int64 
 5   school_holidays         23016 non-null  int64 
 6   warehouse               23016 non-null  object
dtypes: int64(4), object(3)
memory usage: 1.2+ MB


The earliest date in dataset

In [11]:
calendar_df.date.min()

'2016-01-01'

The latest date in dataset

In [12]:
calendar_df.date.max()

'2024-12-31'

##### inventory_df

In [13]:
inventory_df.head()

,unique_id,product_unique_id,name,L1_category_name_en,L2_category_name_en,L3_category_name_en,L4_category_name_en,warehouse
0,5255,2583,Pastry_196,Bakery,Bakery_L2_14,Bakery_L3_26,Bakery_L4_1,Prague_3
1,4948,2426,Herb_19,Fruit and vegetable,Fruit and vegetable_L2_30,Fruit and vegetable_L3_86,Fruit and vegetable_L4_1,Prague_3
2,2146,1079,Beet_2,Fruit and vegetable,Fruit and vegetable_L2_3,Fruit and vegetable_L3_65,Fruit and vegetable_L4_34,Prague_1
3,501,260,Chicken_13,Meat and fish,Meat and fish_L2_13,Meat and fish_L3_27,Meat and fish_L4_5,Prague_1
4,4461,2197,Chicory_1,Fruit and vegetable,Fruit and vegetable_L2_17,Fruit and vegetable_L3_33,Fruit and vegetable_L4_1,Frankfurt_1


In [14]:
inventory_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5432 entries, 0 to 5431
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   unique_id            5432 non-null   int64 
 1   product_unique_id    5432 non-null   int64 
 2   name                 5432 non-null   object
 3   L1_category_name_en  5432 non-null   object
 4   L2_category_name_en  5432 non-null   object
 5   L3_category_name_en  5432 non-null   object
 6   L4_category_name_en  5432 non-null   object
 7   warehouse            5432 non-null   object
dtypes: int64(2), object(6)
memory usage: 339.6+ KB


#### sales_test_df

In [15]:
sales_train_df.head()

,unique_id,date,warehouse,total_orders,sales,sell_price_main,availability,type_0_discount,type_1_discount,type_2_discount,type_3_discount,type_4_discount,type_5_discount,type_6_discount
0,4845,2024-03-10,Budapest_1,6436.0,16.34,646.26,1.00,0.00000,0.0,0.0,0.0,0.15312,0.0,0.0
1,4845,2021-05-25,Budapest_1,4663.0,12.63,455.96,1.00,0.00000,0.0,0.0,0.0,0.15025,0.0,0.0
2,4845,2021-12-20,Budapest_1,6507.0,34.55,455.96,1.00,0.00000,0.0,0.0,0.0,0.15025,0.0,0.0
3,4845,2023-04-29,Budapest_1,5463.0,34.52,646.26,0.96,0.20024,0.0,0.0,0.0,0.15312,0.0,0.0
4,4845,2022-04-01,Budapest_1,5997.0,35.92,486.41,1.00,0.00000,0.0,0.0,0.0,0.15649,0.0,0.0


In [16]:
sales_train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 14 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   unique_id        1048575 non-null  int64  
 1   date             1048575 non-null  object 
 2   warehouse        1048575 non-null  object 
 3   total_orders     1048556 non-null  float64
 4   sales            1048556 non-null  float64
 5   sell_price_main  1048575 non-null  float64
 6   availability     1048575 non-null  float64
 7   type_0_discount  1048575 non-null  float64
 8   type_1_discount  1048575 non-null  float64
 9   type_2_discount  1048575 non-null  float64
 10  type_3_discount  1048575 non-null  float64
 11  type_4_discount  1048575 non-null  float64
 12  type_5_discount  1048575 non-null  float64
 13  type_6_discount  1048575 non-null  float64
dtypes: float64(11), int64(1), object(2)
memory usage: 112.0+ MB


The earliest date in dataset

In [17]:
sales_train_df.date.min()

'2020-08-01'

The latest date in dataset

In [18]:
sales_train_df.date.max()

'2024-06-02'

#### sales_test_df

In [19]:
sales_test_df.head(15)

,unique_id,date,warehouse,total_orders,sell_price_main,type_0_discount,type_1_discount,type_2_discount,type_3_discount,type_4_discount,type_5_discount,type_6_discount
0,1226,2024-06-03,Brno_1,8679.0,13.13,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
1,1226,2024-06-11,Brno_1,8795.0,13.13,0.15873,0.0,0.0,0.0,0.0,0.0,0.0
2,1226,2024-06-13,Brno_1,10009.0,13.13,0.15873,0.0,0.0,0.0,0.0,0.0,0.0
3,1226,2024-06-15,Brno_1,8482.0,13.13,0.15873,0.0,0.0,0.0,0.0,0.0,0.0
4,1226,2024-06-09,Brno_1,8195.0,13.13,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
5,1226,2024-06-06,Brno_1,9538.0,13.13,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
6,1226,2024-06-12,Brno_1,9002.0,13.13,0.15873,0.0,0.0,0.0,0.0,0.0,0.0
7,1226,2024-06-05,Brno_1,8501.0,13.13,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
8,1226,2024-06-04,Brno_1,8492.0,13.13,0.00000,0.0,0.0,0.0,0.0,0.0,0.0
9,1226,2024-06-08,Brno_1,8093.0,13.13,0.00000,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
sales_test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47021 entries, 0 to 47020
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   unique_id        47021 non-null  int64  
 1   date             47021 non-null  object 
 2   warehouse        47021 non-null  object 
 3   total_orders     47021 non-null  float64
 4   sell_price_main  47021 non-null  float64
 5   type_0_discount  47021 non-null  float64
 6   type_1_discount  47021 non-null  float64
 7   type_2_discount  47021 non-null  float64
 8   type_3_discount  47021 non-null  float64
 9   type_4_discount  47021 non-null  float64
 10  type_5_discount  47021 non-null  float64
 11  type_6_discount  47021 non-null  float64
dtypes: float64(9), int64(1), object(2)
memory usage: 4.3+ MB


The earliest date in dataset

In [21]:
sales_test_df.date.min()

'2024-06-03'

The latest date in dataset

In [22]:
sales_test_df.date.max()

'2024-06-16'

#### solution_df

In [23]:
solution_df.head(5)

,id,sales_hat
0,1226_2024-06-03,0
1,1226_2024-06-11,0
2,1226_2024-06-13,0
3,1226_2024-06-15,0
4,1226_2024-06-09,0


In [24]:
solution_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47021 entries, 0 to 47020
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         47021 non-null  object
 1   sales_hat  47021 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 734.8+ KB


#### test_weight_df

In [25]:
test_weights_df.head(5)

,unique_id,weight
0,0,2.535369
1,1,3.888933
2,2,5.885237
3,3,2.139552
4,5,3.021715


In [26]:
test_weights_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5390 entries, 0 to 5389
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   unique_id  5390 non-null   int64  
 1   weight     5390 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 84.3 KB


### Data preprocessing

In [27]:
calendar_df['holiday_name'].value_counts()

Labour Day                                                     63
2nd Christmas Day                                              63
1st Christmas Day                                              63
International womens day                                       63
Christmas Eve                                                  63
New Years Day                                                  63
Easter Monday                                                  42
Good Friday                                                    42
Den boje za svobodu a demokracii                               36
Den vzniku samostatneho ceskoslovenskeho statu                 36
Jan Hus                                                        36
Den ceske statnosti                                            36
Den osvobozeni                                                 36
Cyrila a Metodej                                               36
Whit sunday                                                    27
Whit monda

Translation of some czech holidays

In [28]:
calendar_df['holiday_name'].replace(to_replace=r'Den boje za svobodu a demokracii', value='Struggle for Freedom and Demokracy Day', regex=False)
calendar_df['holiday_name'].replace(to_replace=r'Den vzniku samostatneho ceskoslovenskeho statu', value='Foundation of Independent Czechoslovakia', regex=True)
calendar_df['holiday_name'].replace(to_replace=r'Jan Hus', value='Jan Hus Day', regex=False)
calendar_df['holiday_name'].replace(to_replace=r'Den ceske statnosti', value='St. Wenceslaus Day', regex=False)
calendar_df['holiday_name'].replace(to_replace=r'Den Osvobozeni', value='Victory over Fascism Day', regex=False)
calendar_df['holiday_name'].replace(to_replace=r'Cyrila a Metodej', value='St. Cyril and Metodius Day', regex=False)

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
        ... 
23011    NaN
23012    NaN
23013    NaN
23014    NaN
23015    NaN
Name: holiday_name, Length: 23016, dtype: object

In [29]:
calendar_df.replace({'Den boje za svobodu a demokracii': 10})

,date,holiday_name,holiday,shops_closed,winter_school_holidays,school_holidays,warehouse
0,2022-03-16,NaN,0,0,0,0,Frankfurt_1
1,2020-03-22,NaN,0,0,0,0,Frankfurt_1
2,2018-02-07,NaN,0,0,0,0,Frankfurt_1
3,2018-08-10,NaN,0,0,0,0,Frankfurt_1
4,2017-10-26,NaN,0,0,0,0,Prague_2
...,...,...,...,...,...,...,...
23011,2019-04-20,NaN,1,0,0,0,Munich_1
23012,2018-03-31,NaN,1,0,0,0,Munich_1
23013,2017-04-15,NaN,1,0,0,0,Munich_1
23014,2023-04-08,NaN,1,0,0,1,Munich_1


#### Merging

In [30]:
data = sales_train_df.merge(calendar_df, on='date', how='left').merge(inventory_df, on='unique_id', how='left')

In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7340025 entries, 0 to 7340024
Data columns (total 27 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   unique_id               int64  
 1   date                    object 
 2   warehouse_x             object 
 3   total_orders            float64
 4   sales                   float64
 5   sell_price_main         float64
 6   availability            float64
 7   type_0_discount         float64
 8   type_1_discount         float64
 9   type_2_discount         float64
 10  type_3_discount         float64
 11  type_4_discount         float64
 12  type_5_discount         float64
 13  type_6_discount         float64
 14  holiday_name            object 
 15  holiday                 int64  
 16  shops_closed            int64  
 17  winter_school_holidays  int64  
 18  school_holidays         int64  
 19  warehouse_y             object 
 20  product_unique_id       int64  
 21  name                    object 

Removing redundant columns

In [32]:
data=data.drop(columns='warehouse_y')
data=data.drop(columns='warehouse_x')

In [33]:
data.head(5)

,unique_id,date,total_orders,sales,sell_price_main,availability,type_0_discount,type_1_discount,type_2_discount,type_3_discount,...,shops_closed,winter_school_holidays,school_holidays,product_unique_id,name,L1_category_name_en,L2_category_name_en,L3_category_name_en,L4_category_name_en,warehouse
0,4845,2024-03-10,6436.0,16.34,646.26,1.0,0.0,0.0,0.0,0.0,...,0,0,0,2375,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,Budapest_1
1,4845,2024-03-10,6436.0,16.34,646.26,1.0,0.0,0.0,0.0,0.0,...,0,0,0,2375,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,Budapest_1
2,4845,2024-03-10,6436.0,16.34,646.26,1.0,0.0,0.0,0.0,0.0,...,0,0,0,2375,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,Budapest_1
3,4845,2024-03-10,6436.0,16.34,646.26,1.0,0.0,0.0,0.0,0.0,...,0,0,0,2375,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,Budapest_1
4,4845,2024-03-10,6436.0,16.34,646.26,1.0,0.0,0.0,0.0,0.0,...,0,0,0,2375,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,Budapest_1


#### Date engineering

In [34]:
data['day_of_week'] = pd.to_datetime(data['date']).dt.dayofweek
data['weekend'] = data['day_of_week'].isin([5, 6]).astype(int)
data['holiday'] = data['holiday'].fillna(0).astype(int)

In [35]:
# Rolling
data['rolling_mean_7'] = data.groupby('unique_id')['sales'].shift(1).rolling(window=7).mean()

In [36]:
#Drop rows with NaN values after featue engineering
data_train = data.dropna()

In [37]:
for lag in [1, 7, 30]:
    data[f'sales_lag_{lag}'] = data.groupby('unique_id')['sales'].shift(lag)

In [38]:
data_train.head(5)

,unique_id,date,total_orders,sales,sell_price_main,availability,type_0_discount,type_1_discount,type_2_discount,type_3_discount,...,product_unique_id,name,L1_category_name_en,L2_category_name_en,L3_category_name_en,L4_category_name_en,warehouse,day_of_week,weekend,rolling_mean_7
77,4845,2020-12-24,2998.0,0.0,494.02,1.0,0.0,0.0,0.0,0.0,...,2375,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,Budapest_1,3,0,28.720000
78,4845,2020-12-24,2998.0,0.0,494.02,1.0,0.0,0.0,0.0,0.0,...,2375,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,Budapest_1,3,0,24.617143
79,4845,2020-12-24,2998.0,0.0,494.02,1.0,0.0,0.0,0.0,0.0,...,2375,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,Budapest_1,3,0,20.514286
80,4845,2020-12-24,2998.0,0.0,494.02,1.0,0.0,0.0,0.0,0.0,...,2375,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,Budapest_1,3,0,16.411429
81,4845,2020-12-24,2998.0,0.0,494.02,1.0,0.0,0.0,0.0,0.0,...,2375,Croissant_35,Bakery,Bakery_L2_18,Bakery_L3_83,Bakery_L4_1,Budapest_1,3,0,12.308571


### Explanatory Data Analysis

In [39]:
sales_calendar = sales_train_df.merge(calendar_df, on='date', how='left')


In [40]:
sales_calendar.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7340025 entries, 0 to 7340024
Data columns (total 20 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   unique_id               int64  
 1   date                    object 
 2   warehouse_x             object 
 3   total_orders            float64
 4   sales                   float64
 5   sell_price_main         float64
 6   availability            float64
 7   type_0_discount         float64
 8   type_1_discount         float64
 9   type_2_discount         float64
 10  type_3_discount         float64
 11  type_4_discount         float64
 12  type_5_discount         float64
 13  type_6_discount         float64
 14  holiday_name            object 
 15  holiday                 int64  
 16  shops_closed            int64  
 17  winter_school_holidays  int64  
 18  school_holidays         int64  
 19  warehouse_y             object 
dtypes: float64(11), int64(5), object(4)
memory usage: 1.1+ GB


In [41]:
sales_calendar=sales_calendar.drop(columns='warehouse_y')
sales_calendar=sales_calendar.rename(columns={'warehouse_x':"warehouse"})

In [42]:
# Convert 'date' column to datetime
sales_calendar['date'] = pd.to_datetime(sales_calendar['date'])

# Add 'year' and 'month' columns
sales_calendar['year'] = sales_calendar['date'].dt.year
sales_calendar['month'] = sales_calendar['date'].dt.month
sales_calendar['day']=sales_calendar['date'].dt.month
sales_calendar['day_of_week'] = pd.to_datetime(sales_calendar['date']).dt.dayofweek
sales_calendar['weekend'] = sales_calendar['day_of_week'].isin([5, 6]).astype(int)
sales_calendar['holiday'] = sales_calendar['holiday'].fillna(0).astype(int)

In [55]:
sales_calendar['day_of_week'].value_counts()

4    1059093
3    1057455
2    1055348
5    1049874
1    1048873
0    1037561
6    1031821
Name: day_of_week, dtype: int64

In [43]:
sales_calendar['day_of_week'].value_counts()

4    1059093
3    1057455
2    1055348
5    1049874
1    1048873
0    1037561
6    1031821
Name: day_of_week, dtype: int64

In [44]:
sales_calendar.set_index('date', inplace=True)


In [48]:
sales_calendar['warehouse'].value_counts()

Prague_2       1560384
Prague_3       1483349
Prague_1       1442028
Brno_1         1306032
Budapest_1      999068
Munich_1        314307
Frankfurt_1     234857
Name: warehouse, dtype: int64

In [53]:
prague_1_df = sales_calendar[sales_calendar['warehouse'] == 'Prague_1']

In [54]:
prague_1_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1442028 entries, 2022-03-27 to 2022-07-07
Data columns (total 23 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   unique_id               1442028 non-null  int64  
 1   warehouse               1442028 non-null  object 
 2   total_orders            1442028 non-null  float64
 3   sales                   1442028 non-null  float64
 4   sell_price_main         1442028 non-null  float64
 5   availability            1442028 non-null  float64
 6   type_0_discount         1442028 non-null  float64
 7   type_1_discount         1442028 non-null  float64
 8   type_2_discount         1442028 non-null  float64
 9   type_3_discount         1442028 non-null  float64
 10  type_4_discount         1442028 non-null  float64
 11  type_5_discount         1442028 non-null  float64
 12  type_6_discount         1442028 non-null  float64
 13  holiday_name            61159 non-null    

In [61]:
prague_1_grouped=prague_1_df.groupby(prague_1_df.index).agg({'total_orders':'sum','sales': 'sum','holiday_name': 'first', 'holiday':'first', 'shops_closed':'first', 'winter_school_holidays':'first','school_holidays':'first', 'year':'first', 'month':'first', 'day':'first', 'day_of_week':'first'})

In [62]:
prague_1_grouped.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1402 entries, 2020-08-01 to 2024-06-02
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   total_orders            1402 non-null   float64
 1   sales                   1402 non-null   float64
 2   holiday_name            134 non-null    object 
 3   holiday                 1402 non-null   int32  
 4   shops_closed            1402 non-null   int64  
 5   winter_school_holidays  1402 non-null   int64  
 6   school_holidays         1402 non-null   int64  
 7   year                    1402 non-null   int64  
 8   month                   1402 non-null   int64  
 9   day                     1402 non-null   int64  
 10  day_of_week             1402 non-null   int64  
dtypes: float64(2), int32(1), int64(7), object(1)
memory usage: 126.0+ KB


In [66]:
prague_1_grouped.duplicated()

date
2020-08-01    False
2020-08-02    False
2020-08-03    False
2020-08-04    False
2020-08-05    False
              ...  
2024-05-29    False
2024-05-30    False
2024-05-31    False
2024-06-01    False
2024-06-02    False
Length: 1402, dtype: bool

In [64]:
# Pivot the data to have one column per year
pivoted_data = prague_1_grouped.pivot(index='day', columns='year', values='sales')

# Plot each year's data as a line
plt.figure(figsize=(10, 6))
for year in pivoted_data.columns:
    plt.plot(pivoted_data.index, pivoted_data[year], label=str(year))

plt.title("Sales Trends by Year")
plt.xlabel("Day of Year")
plt.ylabel("Sales")
plt.legend(title="Year")
plt.grid(True)
plt.tight_layout()
plt.show()

ValueError: Index contains duplicate entries, cannot reshape